# VAE Training

## imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

from models.VAE import VariationalAutoencoder
from utils.loaders import load_mnist

In [3]:
# run params
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #

## data

In [4]:

(x_train, y_train), (x_test, y_test) = load_mnist()


## architecture

In [5]:
vae = VariationalAutoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
    , r_loss_factor = 1000
)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [6]:
vae.encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 28, 28, 32)   0           encoder_conv_0[0][0]             
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 14, 14, 64)   18496       leaky_re_lu[0][0]                
____________________________________________________________________________________________

In [7]:
vae.decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 3136)              9408      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 64)        0   

## training

In [8]:
LEARNING_RATE = 0.0005

In [9]:
vae.compile(LEARNING_RATE)

In [10]:
BATCH_SIZE = 32
EPOCHS = 2
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [11]:
vae.train(     
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/2
1875/1875 [==============================] - ETA: 0s - loss: 58.5667 - reconstruction_loss: 55.3597 - kl_loss: 3.2070
Epoch 00001: saving model to run/vae/0002_digits/weights/weights.h5'

Epoch 00001: saving model to run/vae/0002_digits/weights/weights.h5
1875/1875 [==============================] - 72s 39ms/step - loss: 58.5640 - reconstruction_loss: 55.3569 - kl_loss: 3.2072 - lr: 5.0000e-04
Epoch 2/2
1875/1875 [==============================] - ETA: 0s - loss: 51.8131 - reconstruction_loss: 47.8684 - kl_loss: 3.9448
Epoch 00002: saving model to run/vae/0002_digits/weights/weights.h5'

Epoch 00002: saving model to run/vae/0002_digits/weights/weights.h5
1875/1875 [==============================] - 68s 36ms/step - loss: 51.8128 - reconstruction_loss: 47.8677 - kl_loss: 3.9452 - lr: 5.0000e-04
